In [ ]:
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import logging
logging.Logger.disabled=True

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings;
with warnings.catch_warnings():
    warnings.simplefilter("ignore"); 
    import matplotlib.pyplot as plt

import os
import sys
from pathlib import Path
import IPython
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from scipy.fft import fft, fftfreq
import numpy as np
import unittest

import schemdraw
import schemdraw.elements as elm

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

from IPython.display import Javascript
from IPython.display import display, HTML

# amplitude modulation

In amplitude modulation the amplitude (signal strength) of the carrier changes in proportion to the message signal.  






$$
\begin{array}{c}
c(t) = A \sin(2 \pi f_c t)\,
\end{array}
$$




Where A is the amplitude of the carrier singal and the function of f the modulation signal. Amplitude Modulation is a multiplication of the two signals. 

In [ ]:
f1 = 1000
f2 = 200

N = 1000
# sample spacing
T = 1.0 / 20000.0
x = np.linspace(0.0, N*T, N, endpoint=False)

y1 = np.sin(f1 * 2.0 * np.pi * x)
y2 = np.sin(f2 * 2.0 * np.pi * x)
y3 = np.sin(f1 * 2.0 * np.pi * x) * np.sin(f2 * 2.0 * np.pi * x)
y4 = np.sin(f1 * 2.0 * np.pi * x) * np.sin(f2 * 2.0 * np.pi * x) + np.sin(f1 * 2.0 * np.pi * x)

fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, sharex=False, sharey=False, figsize=(8, 6))

#plot multiplication
ax0.plot(x[0:200], y1[0:200], c='Grey', linestyle='--')  # differential output
ax0.plot(x[0:200], y2[0:200], c='Grey')  # differential output
ax0.plot(x[0:200], y3[0:200], c='Red')  # differential output
ax0.set_ylabel('amplitude')
ax0.grid()

y = y3
yf = fft(y)
#print(yf.size)
xf = fftfreq(N, T)[:N//2]

ax1.plot(xf[0:125], 2.0/N * np.abs(yf[0:N//8]))
ax1.grid()
ax1.set_ylabel('frequency')
plt.xticks(np.arange(min(xf[0:250]), max(xf[0:125])+1, 400))

plt.tight_layout()
plt.show()


Here the carrier frequency is 1kHz and the modulation frequency is 200Hz. The carrier frequency is modulated by the modulation frequency. When the modulation signal is negative the resulting signal has a phase shift of 180°. The frequency analysis shows that the two sidebands are created at 800Hz and 1200Hz. The modulation frequency is added and subtracted from the base frequency. This is called a double-sideband with suppressed carrier (DSBSC).

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, sharex=False, sharey=False, figsize=(8, 6))

#plot multiplication with carrier added
ax0.plot(x[0:200], y1[0:200], c='Grey', linestyle='--')  # differential output
ax0.plot(x[0:200], y2[0:200], c='Grey')  # differential output
ax0.plot(x[0:200], y4[0:200], c='Red')  # differential output
ax0.grid()
ax0.set_ylabel('amplitude')

y = y4
yf = fft(y)
xf = fftfreq(N, T)[:N//2]
ax1.plot(xf[0:125], 2.0/N * np.abs(yf[0:N//8]))
ax1.set_ylabel('frequency')
plt.xticks(np.arange(min(xf[0:250]), max(xf[0:125])+1, 400))
ax1.grid()

plt.tight_layout()
plt.show()

When the carrier frequency is added to the result, we see two things. First, the final signal is not phase-shifted when the base signal is negative. Second, the carrier signal is part of the final signal. This is basic amplitude modulation. but notice that the output amplitude is twice the input.

# Long tailed pair

The long-tailed pair or differential amplifier is probably the most widely used circuit building block. For example, the long-tailed pair is the base for op-amps. Also in synthesizer circuits, we see the long-tailed pair a lot for converting CV signals into current. This circuit can be implemented with BJTs or MOSFETs. The differential pair multiplies the voltage difference between the two inputs with the differential gain. The differential gain can be configured with the current in the long tail. The output can be taken either from one side or the difference from both sides. 

In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

Q1 = d.add(elm.BjtNpn( label='$Q_1$', circle=True))
d.add(elm.Line("up"))
d.add(elm.Resistor( "up", label="$R_1$"))
d.add(elm.Vdd(label="+15V"))

IN_X = d.add(elm.Line("left", l=d.unit/2, xy=Q1.base))
d.add(elm.Dot('left', open=True))

L_Q1_emmitter = d.add(elm.Line("down",  xy=Q1.emitter, l=d.unit/2))
d.add(elm.Line("right", l=d.unit*2))
d.add(elm.Line("up", l=d.unit/2))
Q2 = d.add(elm.BjtNpn("right", lftlabel='$Q_2$', anchor="emitter", circle=True, reverse=True))
d.add(elm.Line("up", xy=Q2.collector))
d.add(elm.Resistor( "up", label="$R_2$"))
d.add(elm.Vdd(label="+15V"))

d.add(elm.Line("right", xy=Q2.base, l=d.unit/4))
d.add(elm.Line("down", l=d.unit/4))
d.add(elm.Ground())

d.add(elm.Dot(xy=[L_Q1_emmitter.end[0]+d.unit, L_Q1_emmitter.end[1]]))
R3 = d.add(elm.Resistor( "down", label="$R_3$"))
d.add(elm.Dot())
d.add(elm.Resistor( "down", label="$R_4$"))
d.add(elm.Vss(botlabel="-15V"))

d.add(elm.Line("left", l=d.unit, xy=R3.end))
d.add(elm.Line("down", l=d.unit/2))
Q3 = d.add(elm.BjtNpn("right", rgtlabel='$Q_3$', anchor="emitter", circle=True, flip=True))
d.add(elm.Ground(xy=Q3.collector))
IN_Y = d.add(elm.Line("left", xy=Q3.base, tox=IN_X.end))
d.add(elm.Dot('left', open=True))
d.add(elm.Gap('down', xy=IN_X.end, toy=IN_Y.end, label=['X', 'IN', 'Y']))

d.add(elm.Dot(xy=[Q1.collector[0], Q1.collector[1]+d.unit/3]))
L_OUT_A = d.add(elm.Line("right", l=d.unit*3))
d.add(elm.Dot('right', open=True))
d.add(elm.Dot(xy=[Q2.collector[0], Q2.collector[1]+d.unit/3*2]))
L_OUT_B = d.add(elm.Line("right", tox=L_OUT_A.end))
d.add(elm.Dot('right', open=True))

d.add(elm.Label('right', xy=[L_OUT_A.end[0]+d.unit/4, L_OUT_A.end[1]+d.unit/8], label='OUT'))

d.draw()

In this typical vca configuration, the audio signal is applied to the transistor Q1 where Q2 is grounded. The multiplication factor, or current, is set with the transistor Q3. The output is the difference of OUTa and OUTb (OUTb - OUTa). 

In [ ]:
circuit = Circuit('differential pair')
circuit.include(spice_library['BC547B'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'IN_a', circuit.gnd, 'DC 0V AC 0V SIN(0 25m 1k)')
circuit.V('4', 'IN_b', circuit.gnd, 'DC 0V AC 0V SIN(0 5 100)')

circuit.BJT('1', 'OUT_A', 'IN_a', 'R3_in', model="BC547B")
circuit.BJT('2', 'OUT_B', circuit.gnd, 'R3_in', model="BC547B")
circuit.BJT('3', circuit.gnd, 'IN_b', 'R3_out', model="BC547B")

circuit.R(1, 'OUT_A', '+15V', 15@u_kΩ)
circuit.R(2, 'OUT_B', '+15V', 15@u_kΩ)
circuit.R(3, 'R3_in', 'R3_out', 33@u_kΩ)
R4 = circuit.R(4, 'R3_out', '-15V', 15@u_kΩ)


In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
#ax1_buffer.plot(u_ms(analysis['IN_a'].abscissa), analysis['IN_a'], color='Blue')
ax1_buffer.plot(u_ms(analysis['IN_b'].abscissa), analysis['IN_b'] / 5, color='Blue')
#ax1_buffer.plot(u_ms(analysis['OUT_A'].abscissa), analysis['OUT_A'], color='Grey')
#ax1_buffer.plot(u_ms(analysis['OUT_A'].abscissa), analysis['OUT_B'], color='Grey')
ax1_buffer.plot(u_ms(analysis['OUT_A'].abscissa), analysis['OUT_A'] - analysis['OUT_B'], color='#E97451')
ax1_buffer.legend(('Vin_a [V]', 'Vin_b', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

In [ ]:
from IPython.display import display, HTML
display(HTML('''
<div class="section">
<div class="container">
<div class="content">
<canvas id="myChart" width="600" height="500px"></canvas>
</div>
</div>
</div>
'''))


In [ ]:
import json

data = []
diff = (analysis['OUT_A'] - analysis['OUT_B'])
out = []
labels = []
for item in analysis['IN_b'] :
    value = float(item)
    data.append(value/5)

for item in diff :
    value = float(item)
    out.append(value)
    
for item in analysis['IN_b'].abscissa :
    label = float(item)
    labels.append(label*1000)


The output (red) is the signal multiplied by the input at the long tail. But we see that only the negative signal creates amplification. When the signal is positive the output is silent. This circuit can be useful for a VCA where a DC envelope is applied.

# gilbert cell

Another circuit for the multiplication of two signals is the Gilbert Cell. Although everybody calls it the Gilbert Cell, it is not invented by [Barrie Gilbert](https://en.wikipedia.org/wiki/Barrie_Gilbert). The circuit was first used by Howard Jones in 1963. But Barrie Gilbert invented and augmented it independently and made it a common building block in analog electronics. The Gilbert Cell essentially comprises two differential transistor pairs whose bias current is controlled by one of the input signals.

In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

d += (Q1 := elm.BjtNpn(circle=True).label('$Q_1$'))
d += elm.Line().left().at((Q1, 'base'))
d += (INa := elm.Dot(open=True))
d += elm.Line().up().at((Q1, 'collector'))
d += (R1 := elm.Resistor().up().label('$R_{1}$'))
d += (L1 := elm.Line().right().length(d.unit*2))
d += elm.Dot()
d += elm.Vdd().label('+15V')
d += elm.Line().right().length(d.unit*2)
d += ( R2 := elm.Resistor().down().label('$R_{2}$') )
d += elm.Line().down()
d += (Q4 := elm.BjtNpn(circle=True, anchor='collector').left().flip().label('$Q_4$', loc="lft"))

d += elm.Line().right().length(d.unit/2).at((Q4, 'base'))
d += elm.Line().down().length(d.unit/2)
d += elm.Line().left().length(d.unit*5.5)
d += elm.Line().up().length(d.unit/2)
d += elm.Dot()

d += elm.Dot().at((R1, 'start'))
d += (OUTa := elm.Line().right())
d += elm.Dot(open=True)
d += elm.Dot().at((R2, 'end'))
d += (OUTb := elm.Line().left())
d += elm.Dot(open=True)
d += (G := elm.Gap().left().tox(OUTa.end).label(['b', 'OUT', 'a']))

d += elm.Line().down().at((Q1, 'emitter')).length(d.unit/2)
d += (L2 := elm.Line().right().length(d.unit/2))
d += elm.Dot()
d += elm.Line().down().length(d.unit/2)
d += (Q5 := elm.BjtNpn(circle=True, anchor='collector').right().label('$Q_5$'))
d += elm.Line().down().length(d.unit/2).at((Q5, 'emitter'))
d += elm.Line().right().length(d.unit*1.5)
d += (Lmin := elm.Dot())
d += elm.Resistor().down().label('$R_3$')
d += elm.Vss().label('-15V', loc='bot')

d += elm.Line().right().at(Lmin.end).length(d.unit*1.5)
d += elm.Line().up().length(d.unit/2)
d += (Q6 := elm.BjtNpn(circle=True, anchor='emitter').left().flip().label('$Q_6$', loc="lft"))
d += (L4 := elm.Line().up().length(d.unit/2).at((Q6, 'collector')))
d += elm.Dot()
d += elm.Line().left().length(d.unit/2)
d += elm.Line().up().length(d.unit/2)
d += (Q3 := elm.BjtNpn(circle=True, anchor='emitter').right().label('$Q_3$'))
d += elm.Line().up().at((Q3, 'collector')).length(d.unit/2-d.unit/8)
d += elm.Line().left().length(d.unit*3)
d += elm.Dot()
d += elm.Line().right().length(d.unit/2).at((L4, 'end'))
d += elm.Line().up().length(d.unit/2)

d += elm.Line().at((Q6, 'base')).right().length(d.unit/4)
d += elm.Vdd().label('$V_{bias}$')

d += elm.Line().left().length(d.unit/2).at((Q5, 'base'))
d += (C1 := elm.Capacitor().left().label('$C_1$'))
d += elm.Dot(open=True)
d += (elm.Gap().up().toy(INa.end).label(['x', 'IN', 'y']))
d += elm.Vdd().label('$V_{bias}$').at((C1, 'start'))

d += elm.Line().right().length(d.unit/2).at((L2, 'end'))
d += elm.Line().up().length(d.unit/2)
d += (Q2 := elm.BjtNpn(circle=True, anchor='emitter').left().flip().label('$Q_2$', loc="lft"))
d += elm.Line().up().at((Q2, 'collector')).length(d.unit/2)
d += elm.Line().right().length(d.unit*3)
d += elm.Dot()

d += ( Lgnd := elm.Line().right().at((Q2, 'base')).length(d.unit*0.7))
d += elm.Dot()
d += elm.Line().right().tox(Q3.base)
d += elm.Line().down().at(Lgnd.end).length(d.unit/16)
d += elm.Ground()

d.draw()

It is not so easy to create a discrete Gilbert Cell. All the transistors have to be matched. The most tricky part is the biasing of the input signals. Here the audio signal is biased with a voltage of -7.5V (Vbias). The carrier is biased by the -15V in the long tail. This allows, that the carrier signal can also be a DC signal, like an envelope. This is a very crude implementation of a Gilbert Cell, which most likely is not precise and would add a lot of noise to the output.

In [ ]:
circuit = Circuit('gilbert cell')
circuit.include(spice_library['BC547B'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', 'IN_x', circuit.gnd, 'DC 0V AC 0V SIN(0 10m 1k)')
circuit.V('4', 'IN_y', circuit.gnd, 'DC 0V AC 0V SIN(0 20m 100)')
circuit.V('5', 'Vbias', circuit.gnd, 'DC -7.5V')

circuit.R(1, 'OUT_a', '+15V', 720@u_Ω)
circuit.R(2, 'OUT_b', '+15V', 720@u_Ω)
circuit.R(3, 'R3_in', '-15V', 720@u_Ω)

circuit.R(4, 'R4_out', 'Vbias', 2.2@u_kΩ)
circuit.R(5, 'R5_out', 'Vbias', 2.2@u_kΩ)

circuit.C(1, 'IN_x', 'R4_out', 0.22@u_uF)

# collector, base, emitter
circuit.BJT('1', 'OUT_a', 'IN_y', 'emitter_left', model="BC547B")
circuit.BJT('2', 'OUT_b', circuit.gnd, 'emitter_left', model="BC547B")
circuit.BJT('3', 'OUT_a', circuit.gnd, 'emitter_right', model="BC547B")
circuit.BJT('4', 'OUT_b', 'IN_y', 'emitter_right', model="BC547B")
circuit.BJT('5', 'emitter_left', 'R4_out', 'R3_in', model="BC547B")
circuit.BJT('6', 'emitter_right', 'R5_out', 'R3_in', model="BC547B")

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
simulator.options('INTERP')
analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time (ms)')
ax1_buffer.set_ylabel('IN [V]')
#ax1_buffer.plot(u_ms(analysis['IN_x'].abscissa), analysis['IN_x']*15, color='Blue')
ax1_buffer.plot(u_ms(analysis['IN_y'].abscissa), analysis['IN_y']*8, color='Blue')
ax1_buffer.plot(u_ms(analysis['OUT_a'].abscissa), (analysis['OUT_b'] - analysis['OUT_a']), color='#E97451')
ax1_buffer.legend(('Vin_y [V]', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

The plot shows, that the Gilbert Cell does work as excpected and creates real amplitude modulation with double sideband and supressed carrier.

# Analog Multiplier

We saw that multiple types of multiplier circuits behave differently. Of course, this is a well-defined behaviour. The multipliers are classified in three types. The types are called quadrants. In the following table the different quadrants are listed and for which signal type they can be used.

|Type|X [V]| Y  [V]| Out [V]|
|--- |---  |---    |---     |
|1-Quadrant|Unipolar|Unipolar|Unipolar|
|2-Quadrant|Bipolar|Unipolar|Bipolar|
|4-Quadrant|Bipolar|Bipolar|Bipolar|

The simulation showed that the long-tailed pair is a 2-quadrant multiplier. The X signal can be an audio signal but the Y signal has to be unipolar, for example, an envelope. In contrast, the Gilbert Cell is a four-quadrant multiplier and allows both signals to be bipolar. The Gilbert Cell can be used as a Ring Modulator. of course, a 4-quadrant multiplier also works correctly when one signal is unipolar. But the output signal would be the same as with a 2-quadrant multiplier. 

# conclusion

There are different circuits for multiplication of signals. The choice of the circuit depends on the nature of the signal and the required functions we want to apply to them. Here only circuits using BJT's are shown. The downside of these circuits is that all the transistors have to be properly matched. In my experience, they are also very sensitive to noise. My take on a discrete Gilbert Cell produced to much noise to be useful. There are also integrated circuits which implement a VCA or multiplier available. For a simple VCA the [LM13700](https://www.ti.com/lit/ds/symlink/lm13700.pdf) is a good choice. There are also integrated circuits available which implement the Gilbert Cell. One example is the [AD633](https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf) from Analog Devices. This chip is very handy, it offers the right input and output impedances. No buffering or biasing of the signals is necessary. It is a very precise multiplier, which of course has some price. 

# references

* [LM13700](https://www.ti.com/lit/ds/symlink/lm13700.pdf) Datasheet
* [AD633](https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf) Datasheet
* VCA Techniques Investigated by [Rod Elliott (ESP)](https://sound-au.com/articles/vca-techniques.html)